# Solving Sudoku

The primary description of this coursework is available on the CM20252 Moodle page. This is the Jupyter notebook you must complete and submit to receive marks. 

** You must follow all instructions given in this notebook. **

Restart the kernel and run all cells before submitting the notebook. This will guarantee that we will be able to run your code for testing.

Remember to save your work regularly. 

## Getting Started

You already know that you will be writing a Sudoku solver. You need to implement your solver in Python in this notebook. You can use any of the appropriate problem-solving techniques discussed in the lectures. You are encouraged to modify the basic algorithms. Be creative. 

You will be given Sudoku puzzles that either have a single solution or no solution. You will need to identify the solution, if there is one.

Below is a sample puzzle along with its solution. 

<img src="images/Sudoku_unsolved.png" style="width: 200px;"/>
<img src="images/Sudoku_solved.png" style="width: 200px;"/>


## Sample Sudokus

You can test your code on a set of 20 sample Sudoku puzzles. This set is similar to the test set that will be used to assess your work. 

The following code will load 20 Sudoku puzzles and their solutions into two `20x9x9` numpy arrays. Empty cells are designated by zeros. 

In [1]:
import numpy as np
import time

# Load sudokus
sudokus = np.load("data/sudokus.npy")
print("Shape of one sudoku array:", sudokus[0].shape, ". Type of array values:", sudokus.dtype)

# Load solutions
solutions = np.load("data/sudoku_solutions.npy")
print("Shape of one sudoku solution array:", solutions[0].shape, ". Type of array values:", solutions.dtype, "\n")

# Print the first sudoku...
print("Sudoku #1:")
print(sudokus[0], "\n")

# ...and its solution
print("Solution of Sudoku #1:")
print(solutions[0])

Shape of one sudoku array: (9, 9) . Type of array values: float64
Shape of one sudoku solution array: (9, 9) . Type of array values: float64 

Sudoku #1:
[[0. 0. 4. 0. 8. 3. 0. 0. 2.]
 [0. 5. 1. 0. 0. 4. 3. 0. 0.]
 [0. 0. 0. 0. 9. 6. 7. 1. 0.]
 [1. 2. 0. 8. 0. 0. 0. 0. 6.]
 [0. 4. 0. 0. 0. 0. 5. 0. 0.]
 [8. 3. 0. 6. 0. 7. 9. 0. 0.]
 [0. 6. 0. 3. 0. 9. 0. 4. 0.]
 [0. 0. 7. 0. 0. 0. 2. 0. 5.]
 [0. 9. 0. 0. 5. 0. 8. 0. 3.]] 

Solution of Sudoku #1:
[[9. 7. 4. 1. 8. 3. 6. 5. 2.]
 [6. 5. 1. 2. 7. 4. 3. 8. 9.]
 [2. 8. 3. 5. 9. 6. 7. 1. 4.]
 [1. 2. 9. 8. 3. 5. 4. 7. 6.]
 [7. 4. 6. 9. 1. 2. 5. 3. 8.]
 [8. 3. 5. 6. 4. 7. 9. 2. 1.]
 [5. 6. 8. 3. 2. 9. 1. 4. 7.]
 [3. 1. 7. 4. 6. 8. 2. 9. 5.]
 [4. 9. 2. 7. 5. 1. 8. 6. 3.]]


## Your code ##

Define a function called `sudoku_solver()` that takes **one** Sudoku puzzle (a $9 \times 9$ numpy array) as input and returns the solved Sudoku as a $9 \times 9$ numpy array. Note that the test set may contain invalid Sudokus, that is, Sudokus with no solution. In such a case, your function should return a $9 \times 9$ numpy array whose values are all equal to `-1`.  

You may use more than one cell to write your code (but this is not necessary).

In [2]:
invalidSudoku = np.full((9,9), -1)



def permutation(lst): #Functions grabbed from https://www.geeksforgeeks.org/generate-all-the-permutation-of-a-list-in-python/ 27/02/2019 11:22
  
    # If lst is empty then there are no permutations 
    if len(lst) == 0: 
        return [] 
  
    # If there is only one element in lst then, only 
    # one permuatation is possible 
    if len(lst) == 1: 
        return [lst] 
  
    # Find the permutations for lst if there are 
    # more than 1 characters 
  
    l = [] # empty list that will store current permutation 
  
    # Iterate the input(lst) and calculate the permutation 
    for i in range(len(lst)): 
       m = lst[i] 
  
       # Extract lst[i] or m from the list.  remLst is 
       # remaining list 
       remLst = lst[:i] + lst[i+1:] 
  
       # Generating all permutations where m is first 
       # element 
       for p in permutation(remLst): 
           l.append([m] + p) 
    return l



def getRow(sudoku, a):
    return sudoku[a]

def setRow(sudoku, a, row):
    sudoku[a] = row
    return sudoku
    
    

def getColumn(sudoku, a):
    column = np.zeros(9)
    
    for i in range(9):
        column[i] = getRow(sudoku, i)[a]
        
    return column

def setColumn(sudoku, a, col):
    for i in range(9):
        sudoku[i][a] = col[i]
    
    return sudoku

def getSquare(sudoku, row, column):
    square = np.zeros(9)
    
    count = 0
    for i in range(3):
        ourRow = getRow(sudoku, row*3 + i)
        
        for j in range(3):
            square[count] = ourRow[column*3 + j]
            count = count + 1
            
    return square

def setSquare(sudoku, row, column, square):
    
    count = 0
    for i in range(3):
        for j in range(3):
            sudoku[row*3 + i][column*3 + j] = square[count]
            count = count + 1
            
    return sudoku


            
def countEntries(array):
    count = 0
    
    for i in range(9):
        if array[i] > 0:
            count = count + 1
    
    return count

def mostSquareEntries(sudoku):
    bestSquare = False
    bestSquareCount = 0
    I, J = 0,0
    
    for i in range(3):
        for j in range(3):
            tempSquare = getSquare(sudoku, i, j)
            tempCount = countEntries( tempSquare )
            if tempCount > bestSquareCount:
                bestSquare = tempSquare.copy()
                bestSquareCount = tempCount
                I = i
                J = j
                
    return bestSquare, I, J

def emptyInsert(array, fill): #Inserts 'num' into array at the shift'th empty position, if it exsits.
    count = 0
    for i in range(9):
        if array[i] == 0:
            array[i] = fill[count]
            count = count + 1
            
    return array

def permuteSquare(sudoku, row, col):
    square = getSquare(sudoku, row, col).astype(int).copy()
    allNumbers = np.zeros(10, dtype=int)
    
    numbersLeft = np.zeros( 9-countEntries(square), dtype=int )
    n = numbersLeft.size
    
    for i in square:
        allNumbers[i] = 1
        
    j = 0
    for i in range(1,10):
        if allNumbers[i] == 0:
            numbersLeft[j] = i
            j = j + 1
            
    if numbersLeft.size == 0:
        return sudoku
            

    ourSize = np.math.factorial(n)
    boards = [sudoku.copy()]*ourSize
    permutations = permutation(numbersLeft.tolist())
    validBoards = [0]*ourSize
    validCounter = 0
    for i in range(ourSize):
        board = boards[i].copy()
        newSquare = getSquare(board, row, col).copy()
        newSquare = emptyInsert(newSquare, permutations[i])
        board = setSquare(board, row, col, newSquare)
        
        if isValidBoard(board):
            validBoards[validCounter] = board.copy()
            validCounter = validCounter + 1
        
        boards[i] = board
        i = i + 1

            
    return validBoards[:validCounter]



def containsDuplicates(array):
    check = [-1]*10
    array = array.astype(int)
    
    for i in array:
        if i != 0:
            if check[i] == -1:
                check[i] = 1
            else:           #We've already had this number appear.
                return True
        
    return False

def isValidBoard(sudoku):
    for i in range(9):
        if containsDuplicates( getRow(sudoku, i) ) or containsDuplicates( getColumn(sudoku, i) ):
            return False
    
    for i in range(3):
        for j in range(3):
            if containsDuplicates( getSquare(sudoku, i, j) ):
                return False
            
    return True
        
            
    

def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """
    
    #Lets get our best square, then get a valid state space for it.
    baseSquare, R, C = mostSquareEntries(sudoku)
    validPermutations = permuteSquare(sudoku, R, C)
    
    #Now using these permutations, lets get all the permutations after we permute the 2 squares on the same column
    for i in range(3):
        if not i == R:
            newPerms = [0]*1000
            j = 0
            for board in validPermutations:
                validPerms = permuteSquare(board, i, C)
                for perm in validPerms:
                    newPerms[j] = perm.copy()
                    j = j + 1
                    
            validPermutations = newPerms[:j]
            
            
    for k in range(3):
        if not k == C:
            for i in range(3):
                newPerms = [0]*1000
                j = 0
                for board in validPermutations:
                    validPerms = permuteSquare(board, i, k)
                    for perm in validPerms:
                        newPerms[j] = perm.copy()
                        j = j + 1

                validPermutations = newPerms[:j]
                
    
    #print(validPermutations)
        
    
    
    solved_sudoku = invalidSudoku.copy()
    if len(validPermutations) > 0:
        solved_sudoku = validPermutations[0]
    
    return solved_sudoku


#print(sudokus[0])
#print(getSquare(sudokus[0], 0, 0))
#print(mostSquareEntries(sudokus[0]))
#isValidBoard(sudokus[0])
#sudoku_solver(sudokus[0])


## Testing your code

You can test your code on the sudoku puzzles that we have provided in the following cell. This will work only if all of your code is above the test cell. Otherwise, the test cell does not have access to the `sudoku_solver()` function. Before you submit, please comment out any code that you used to test your function on the training puzzles.

In [3]:
'''# Uncomment the following block to test your code. Comment it out again before submitting.

t = time.time()
print("Start: ", t)
for i in range(len(sudokus)):
    sudoku = sudokus[i].copy()
    print("This is sudoku number", i)
    print(sudoku)
    your_solution = sudoku_solver(sudokus[i])
    print("This is your solution for sudoku number", i)
    print(your_solution)
    print("Is your solution correct?")
    print(np.array_equal(your_solution, solutions[i]))
    print("Elapsed: ", time.time() - t, "s")

'''

Start:  1552328507.8747027
This is sudoku number 0
[[0. 0. 4. 0. 8. 3. 0. 0. 2.]
 [0. 5. 1. 0. 0. 4. 3. 0. 0.]
 [0. 0. 0. 0. 9. 6. 7. 1. 0.]
 [1. 2. 0. 8. 0. 0. 0. 0. 6.]
 [0. 4. 0. 0. 0. 0. 5. 0. 0.]
 [8. 3. 0. 6. 0. 7. 9. 0. 0.]
 [0. 6. 0. 3. 0. 9. 0. 4. 0.]
 [0. 0. 7. 0. 0. 0. 2. 0. 5.]
 [0. 9. 0. 0. 5. 0. 8. 0. 3.]]
This is your solution for sudoku number 0
[[9. 7. 4. 1. 8. 3. 6. 5. 2.]
 [6. 5. 1. 2. 7. 4. 3. 8. 9.]
 [2. 8. 3. 5. 9. 6. 7. 1. 4.]
 [1. 2. 9. 8. 3. 5. 4. 7. 6.]
 [7. 4. 6. 9. 1. 2. 5. 3. 8.]
 [8. 3. 5. 6. 4. 7. 9. 2. 1.]
 [5. 6. 8. 3. 2. 9. 1. 4. 7.]
 [3. 1. 7. 4. 6. 8. 2. 9. 5.]
 [4. 9. 2. 7. 5. 1. 8. 6. 3.]]
Is your solution correct?
True
Elapsed:  3.8520126342773438 s
This is sudoku number 1
[[0. 0. 0. 0. 6. 0. 2. 8. 0.]
 [7. 0. 9. 0. 0. 1. 0. 0. 0.]
 [8. 6. 0. 3. 2. 0. 0. 7. 4.]
 [9. 0. 0. 0. 4. 0. 5. 1. 0.]
 [0. 0. 7. 1. 9. 0. 3. 4. 0.]
 [0. 0. 3. 0. 0. 6. 0. 0. 2.]
 [0. 0. 2. 9. 7. 0. 0. 0. 0.]
 [3. 0. 0. 8. 0. 0. 9. 0. 5.]
 [5. 0. 0. 0. 0. 0. 0. 2. 1.]]
This is 

This is your solution for sudoku number 13
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
True
Elapsed:  32.57145857810974 s
This is sudoku number 14
[[2. 4. 0. 6. 9. 0. 5. 0. 0.]
 [3. 0. 0. 0. 0. 0. 2. 0. 0.]
 [6. 1. 0. 0. 0. 4. 8. 0. 7.]
 [0. 0. 0. 0. 3. 8. 0. 2. 0.]
 [0. 0. 0. 0. 5. 0. 7. 6. 0.]
 [1. 2. 4. 9. 7. 0. 3. 0. 0.]
 [0. 9. 8. 5. 6. 0. 0. 7. 0.]
 [0. 0. 0. 2. 0. 0. 6. 0. 0.]
 [0. 3. 0. 0. 0. 1. 0. 0. 4.]]
This is your solution for sudoku number 14
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solutio

KeyboardInterrupt: 

## How we will test your code

We will test your code using the hidden tests in the following cell. Specifically, the hidden tests will test your `sudoku_solver()` function on a different set of 20 sudoku puzzles of similar difficulty. **Make sure all of your code is above our test cell. ** Otherwise, the test cell will not have access to the sudoku_solver() function and you will receive zero marks.

## IMPORTANT: How to submit

If any of the following instructions is not clear, please ask your tutors well ahead of the submission deadline.

#### Before you submit
- Restart the kernel (_Kernel $\rightarrow$ Restart & Run All_) and make sure that you can run all cells from top to bottom without any errors.
- Make sure that the test cell has access to the `sudoku_solver()` function that you defined and make sure that this function returns the solved Sudoku in the correct data type and shape.
- Please comment out any code that you used to test your function on the training puzzles.
- Make sure that your code is written in Python 3 (and not in Python 2!). You can check the Python version of the current session in the top-right corner below the Python logo.

#### Submission file
- Please upload to Moodle a single Jupyter notebook file called "ai3_sudoku.ipynb". Do __not__ compress/zip your Jupyter notebook file.
- Do not include __any__ identifying information. Marking is anonymous.

In [ ]:
# This is a TEST CELL. Do not delete or change. All of your code must be written above this cell. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 

In [ ]:
# This is a TEST CELL. Do not delete or change. 